# Hourly Bike Sharing Data:
This notebook will:
1. Load and explore `hour.csv` (hour-level bike sharing data).
2. Perform data quality checks across four dimensions:
   - Completeness
   - Uniqueness
   - Accuracy
   - Consistency
   - Validity
3. Resolve any data quality issues.
4. Transform the cleaned data into tables for a star schema:
   - `dim_time` (hour dimension)
   - `fact_hourly_rentals`
5. Save the resulting tables as CSV files.

## 1. Imports and Load Hourly Data

In [11]:
import pandas as pd

# Load the hour-level data
df_hour = pd.read_csv('Dataset/hour.csv', parse_dates=['dteday'])
display(df_hour.head())

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## 2. Data Overview

In [12]:
df_hour.info()
display(df_hour.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     17379 non-null  int64         
 1   dteday      17379 non-null  datetime64[ns]
 2   season      17379 non-null  int64         
 3   yr          17379 non-null  int64         
 4   mnth        17379 non-null  int64         
 5   hr          17379 non-null  int64         
 6   holiday     17379 non-null  int64         
 7   weekday     17379 non-null  int64         
 8   workingday  17379 non-null  int64         
 9   weathersit  17379 non-null  int64         
 10  temp        17379 non-null  float64       
 11  atemp       17379 non-null  float64       
 12  hum         17379 non-null  float64       
 13  windspeed   17379 non-null  float64       
 14  casual      17379 non-null  int64         
 15  registered  17379 non-null  int64         
 16  cnt         17379 non-

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2012-01-02 04:08:34.552045568,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
min,1.0000,2011-01-01 00:00:00,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2011-07-04 00:00:00,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,2012-01-02 00:00:00,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,2012-07-02 00:00:00,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,2012-12-31 00:00:00,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000
std,5017.0295,NaN,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599


## 3. Data Quality Checks

### 3.1 Completeness
Check for missing values in each column.

In [13]:
display(df_hour.isnull().sum())

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

### 3.2 Uniqueness
Check for duplicate rows at the record level.

In [14]:
print('Duplicate rows:', df_hour.duplicated().sum())

Duplicate rows: 0


### 3.3 Accuracy
Verify numeric ranges and code validity.

In [15]:
accuracy_checks = {
    'hour_range': df_hour['hr'].between(0,23).all(),
    'temp_norm_range': df_hour['temp'].between(0,1).all(),
    'hum_norm_range': df_hour['hum'].between(0,1).all(),
    'windspeed_norm_range': df_hour['windspeed'].between(0,1).all(),
    'season_codes': set(df_hour['season'].unique()), # four unique seasons?
    'weather_codes': set(df_hour['weathersit'].unique()) # four unique weather conditions?
}
display(accuracy_checks)

{'hour_range': np.True_,
 'temp_norm_range': np.True_,
 'hum_norm_range': np.True_,
 'windspeed_norm_range': np.True_,
 'season_codes': {np.int64(1), np.int64(2), np.int64(3), np.int64(4)},
 'weather_codes': {np.int64(1), np.int64(2), np.int64(3), np.int64(4)}}

### 3.4 Consistency
Ensure categorical codes match expected mappings.

In [16]:
print('Seasons valid:', set(df_hour['season'].unique()) <= {1,2,3,4})
print('Weather valid:', set(df_hour['weathersit'].unique()) <= {1,2,3,4})

Seasons valid: True
Weather valid: True


### 3.5 Validity
Check that dates and hours fall within expected ranges.

In [17]:
print('Date range:', df_hour['dteday'].min(), 'to', df_hour['dteday'].max())
print('Hour range:', df_hour['hr'].min(), 'to', df_hour['hr'].max())

Date range: 2011-01-01 00:00:00 to 2012-12-31 00:00:00
Hour range: 0 to 23


## 4. Transform to Star Schema

### 4.1 Build `dim_time`

In [19]:
dim_time = pd.DataFrame({'hour_of_day': range(24)})
dim_time['time_label'] = dim_time['hour_of_day'].astype(str).str.zfill(2) + ':00'
display(dim_time.head())

,hour_of_day,time_label
0,0,00:00
1,1,01:00
2,2,02:00
3,3,03:00
4,4,04:00


### 4.2 Build `fact_hourly_rentals`

In [22]:
fact_hourly = pd.DataFrame({
    'date_key': df_hour['dteday'].dt.strftime('%Y%m%d').astype(int),
    'hour_of_day': df_hour['hr'],
    'season_id': df_hour['season'],
    'weather_id': df_hour['weathersit'],
    'casual_count': df_hour['casual'],
    'registered_count': df_hour['registered'],
    'total_count': df_hour['cnt'],
    'temp_c': df_hour['temp'] * (39 - (-8)) + (-8),
    'atemp_c': df_hour['atemp'] * (50 - (-16)) + (-16),
    'humidity_pct': df_hour['hum'] * 100,
    'windspeed_mph': df_hour['windspeed'] * 67
})
display(fact_hourly.head())

,date_key,hour_of_day,season_id,weather_id,casual_count,registered_count,total_count,temp_c,atemp_c,humidity_pct,windspeed_mph
0,20110101,0,1,1,3,13,16,3.28,3.0014,81.0,0.0
1,20110101,1,1,1,8,32,40,2.34,1.9982,80.0,0.0
2,20110101,2,1,1,5,27,32,2.34,1.9982,80.0,0.0
3,20110101,3,1,1,3,10,13,3.28,3.0014,75.0,0.0
4,20110101,4,1,1,0,1,1,3.28,3.0014,75.0,0.0


## 5. Save All Tables to CSV

In [ ]:
dim_time.to_csv('dim_time.csv', index=False)
fact_hourly.to_csv('fact_hourly_rentals.csv', index=False)
print('All tables saved.')

All tables saved.
